In [1]:
# Since I'm using colabs, we need actually implement all these into tensorflow modules so that we can use the gpu.
# %tensorflow_version 1.x
file = open('Tang_poems_utf_8.txt')
text = file.read()
file.close()
import matplotlib.pyplot as plt
## split and create regular poems
import re
import numpy as np
import os
from collections import defaultdict
import numpy as np
import tensorflow as tf

pattern = u'卷.[0-9]+'
poems = re.split(pattern, text)[1:]
regular_poems = []
regular_title = []
corpus = []
N = 100 # total number of samples
stopwords = re.compile(u'而|何|乎|乃|其|且|然|若|所|为|焉|也|以|矣|于|之|则|者|与|欤|因|-')
for poem in poems:
    tmp_poem = poem.strip('\n\u3000\u3000◎')
    tmp_poem = tmp_poem.replace('\u3000\u3000','').split('\n')
    regular_title.append(tmp_poem[0])
    regular_poems.append('\n'.join(tmp_poem[1:]))
    tmp_poem=''.join(tmp_poem[1:]).replace('\n',"").replace('。','').replace('，','').replace('：','').replace('；','').replace('？','').replace('！','').replace('（[.*]*?）','')
    corpus.append(list(stopwords.sub('',tmp_poem)))
    ## Word frequency
    
#corpus=corpus
# corpus=corpus
corpus = corpus[:N]
# develop based on this part
# create a vocabulary
words = []
for poem in corpus:
    for word in poem:
        words.append(word)
words = set(words)
# create map between word and integers
word2int = {}
int2word = {}
vocab_size = len(words)  # gives the total number of unique words
for i, word in enumerate(words):
    word2int[word] = i
    int2word[i] = word
sentences = corpus
# capture each word and their neighborhood
data = []
WINDOW_SIZE = 2
n_count = 0
for sentence in sentences:
    for word_index, word in enumerate(sentence):
        for nb_word in sentence[max(word_index - WINDOW_SIZE, 0):min(word_index + WINDOW_SIZE, len(sentence)+1)]:
            if nb_word != word:
                data.append([word, nb_word])
# BUT actually we don't need this, because we consider each character in the poem as a word, that is to say, a vector
# convert to unit vector.


def unit_vectorization(data_point_index, vocab_size):
    temp = np.zeros(vocab_size)
    temp[data_point_index] = 1
    return temp

vocab_size = len(words)
x_train = []
y_train = []
for data_word in data:
    x_train.append(unit_vectorization(word2int[data_word[0]], vocab_size))
# convert to nparray
x_train = np.asarray(x_train)
for data_word in data:
    y_train.append(unit_vectorization(word2int[data_word[1]], vocab_size))
y_train = np.asarray(y_train)

# convert to nparray
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
# make the tensorflow model
x = tf.placeholder(tf.float32, [None, vocab_size],name="x")
y_label = tf.placeholder(tf.float32, shape=(None, vocab_size),name="y_label")

EMBEDDING_DIM = 5
W1 = tf.Variable(tf.random_normal([vocab_size, EMBEDDING_DIM]),name="W1")
# noise ### this line claims a variable of tf type which is essentially a 1x5 matrix
b1 = tf.Variable(tf.random_normal([EMBEDDING_DIM]),name="b1")
hidden_representation = tf.add(tf.matmul(x, W1), b1)
W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, vocab_size]),name="W2")
b2 = tf.Variable(tf.random_normal([vocab_size]),name="b2")
prediction = tf.nn.softmax(tf.add(tf.matmul(hidden_representation, W2), b2))
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)  # make sure you do this!
# define the loss function:
print('Preparation Done')

/home/hsianktin/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hsianktin/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/hsianktin/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/hsianktin/.local/lib/python3.6/site-packages/tensorflow/python/framework/dty

In [2]:
# define the loss function:
cross_entropy_loss = tf.reduce_mean(-tf.reduce_sum(
    y_label * tf.log(prediction), reduction_indices=[1]))
# define the training step:
train_step = tf.train.GradientDescentOptimizer(
    0.1).minimize(cross_entropy_loss)
n_iters = 301 # Even on google colab implementing 10000 iterations takes a very looooong time
print('Start calculating')
# train for n_iter iterations
saver = tf.train.Saver()
checkpoint_path = "./ckpt/w2v" 
count = 0
for _ in range(n_iters):
    count += 1
    sess.run(train_step, feed_dict={x: x_train, y_label: y_train})
    print(str(count)+'th iteration, loss is : ', sess.run(cross_entropy_loss,
                                 feed_dict={x: x_train, y_label: y_train}))
    if count % 100 == 0:
      save_path = saver.save(sess,checkpoint_path,global_step=count)


Start calculating
1th iteration, loss is :  14.2352705
2th iteration, loss is :  13.458983
3th iteration, loss is :  12.869039
4th iteration, loss is :  12.39673
5th iteration, loss is :  12.008874
6th iteration, loss is :  11.686412
7th iteration, loss is :  11.416661
8th iteration, loss is :  11.190302
9th iteration, loss is :  11.0000515
10th iteration, loss is :  10.840038
11th iteration, loss is :  10.70538
12th iteration, loss is :  10.592067
13th iteration, loss is :  10.496706
14th iteration, loss is :  10.416428
15th iteration, loss is :  10.348841
16th iteration, loss is :  10.291911
17th iteration, loss is :  10.243935
18th iteration, loss is :  10.203472
19th iteration, loss is :  10.169301
20th iteration, loss is :  10.140401
21th iteration, loss is :  10.115904
22th iteration, loss is :  10.095107
23th iteration, loss is :  10.077386
24th iteration, loss is :  10.062241
25th iteration, loss is :  10.049247
26th iteration, loss is :  10.038052
27th iteration, loss is :  10

In [4]:
# output
vectors = sess.run(W1 + b1)


def word_vec(word):
    v_w = vectors[word2int[word]]
    return v_w


def word_sim(word, top_n):
    w1_index = word2int[word]
    v_w1 = vectors[w1_index]
    word_sim = {}
    for i in range(vectors.shape[0]):
        v_w2 = vectors[i]
        theta_num = np.dot(v_w1, v_w2)
        theta_den = np.linalg.norm(v_w1) * np.linalg.norm(v_w2)
        theta = theta_num / theta_den
        word = int2word[i]
        word_sim[word] = theta
    words_sorted = sorted(word_sim.items(), key=lambda x: x[1], reverse=True)
    return words_sorted[1:top_n + 1]



filename_w2v = './output/w2v_matrix_100.txt'
with open(filename_w2v, 'w') as f:
    f.write("字\t对应向量\t训练样本数"+str(N))
    for x in words:
        f.write("\n"+x+"\t"+str(word_vec(x)))
filename_w2v_sample = "./output/w2v_matrix_100_sample.txt"
sample = ['思', '悲', '忧', '愁', '怒', '惧', '乐']
with open(filename_w2v_sample, 'w') as f:
    for mu in sample:
        f.write(mu+": " + "、".join([y[0] for y in word_sim(mu, 7)])+"\n")

print('Files are saved')

KeyError: '怒'